# Application 02: Simulating Depth of Field using OpenCV and Gradio



To simulate the focal properties of a real camera, we utilize a depth map. Depth of Field (DoF) creates sharp focus on the focal plane at a specific distance while keeping other areas blurred. It defines the range between the nearest and farthest objects that appear acceptably sharp in an image.




In [ ]:
# @title _
from IPython.display import HTML, display

html_code = """
<style>
  /* Hide the code cell itself in Colab */
  div.cell.code_cell {
      display: none;
  }
</style>

<div style="text-align: center;">
  <figure style="display: inline-block;">
    <video id="autoplayVideo" autoplay loop muted playsinline width="1080">
      <source src="https://learnopencv.com/wp-content/uploads/2025/01/Depth-of-filed-output-DepthPro-Output-Monocular-depth-applications.webm" type="video/webm">
    </video>
    <figcaption>Simulating Depth of Field Effect with OpenCV</figcaption>
  </figure>
</div>

<script>
  document.addEventListener("DOMContentLoaded", function() {
    var video = document.getElementById("autoplayVideo");
    video.play();
  });
</script>
"""

display(HTML(html_code))


# 1. Introduction

Depth of Field (DoF) is a fundamental concept in photography and cinematography that determines which parts of an image appear sharp and which parts are blurred. In professional cameras, several key factors control DoF, such as the lens aperture, focal length, sensor size, and perhaps most importantly, the **distance between your subject and the camera**.

  

You’ve probably noticed that portrait shots often showcase a beautifully blurred background where only a small portion of the image is in focus—this is achieved using a shallow depth of field. Conversely, landscape shots typically feature everything sharply focused, a deep DoF keeps everything in focus thanks to a deep depth of field.


<div style="text-align: center;">
    <img src="https://learnopencv.com/wp-content/uploads/2025/04/depth-of-field-and-focusing-distance.jpg"
         alt="Depth of Field and Focusing Distance">
    <p style="font-size: 14px; color: gray;">Source: <a href="https://capturetheatlas.com/depth-of-field-photography/" target="_blank">capturetheatlas.com</a></p>
</div>


**Factors Affecting DoF**

- **Aperture**: A wider aperture (lower f-stop) results in a shallower DoF, while a narrower aperture (higher f-stop) increases DoF.  

- **Subject to Camera Distance**: A wider aperture (low f-stop number) gives you a shallow DoF with creamy background blur, perfect for portraits. A narrow aperture (higher f-stop number) keeps everything sharp, ideal for landscapes.
  
- **Lens Focal Length**: Longer lenses (telephoto) deliver a shallower DoF, making backgrounds more pleasantly blurred, whereas shorter lenses (wide-angle) keep a broader area in focus.

- **Sensor Size**: Cameras with larger sensors naturally create shallower depth-of-field effects compared to smaller sensors like those found in smartphones.  


### Simulating Depth of Field Using Image Processing  

In this notebook of Depth Module, we aim to **simulate the effect of subject-to-camera distance** using computational techniques. Instead of relying on physical optics, we will generate a **depth-aware blur effect** using the following steps:  

1. **Depth Estimation**: As usual we will first obtain a depth map from a single image using a monocular depth estimation model like DepthPro. This depth map serves as a proxy for the subject-to-camera distance.  

2. **Depth-Based Blurring**: Using the depth map, we will apply a variable Gaussian blur, selectively defocusing parts of the image to mimic how real-world depth of field behaves. Objects further from your chosen focal point will softly blur, highlighting the intended subject.  

While this method won’t perfectly replicate real-world optics, it allows us to create a convincing DoF effect using only computational techniques. This approach is widely used in computational photography and artificial bokeh effects in smartphone cameras.  

If you're someone who already loves photography and understands manual camera settings, you’ll absolutely love this process! **It's much like finely adjusting the focus ring on your DSLR lens to get that perfect shot frozen in time, with subjects in sharp focus.** 📸  





To know the nitty gritty details of Depth of Field in Professional Photography, check out this excellent [guide](https://photographylife.com/what-is-depth-of-field).





## 2. Depth Prediction - Depth Pro Setup

In [ ]:
%%capture
!git clone https://github.com/apple/ml-depth-pro.git
%cd ml-depth-pro
!pip install -e .

In [ ]:
# Download the model checkpoint from huggingface
%%capture
!pip install huggingface-hub
!pip install gradio
!huggingface-cli download --local-dir checkpoints apple/DepthPro
%cd ..

In [ ]:
# To avoid depth pro import errors
!pip install numpy==1.26.4

In [ ]:
# Check numpy version - To work as expected it should be 1.26.4
import numpy as np
np.__version__

'2.0.2'

📌 **Restart Session**

#### Import Dependencies

In [ ]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
import time
import torch

# Add the src path for depth_pro
sys.path.append('ml-depth-pro/src')
!ln -s ml-depth-pro/checkpoints ./checkpoints # create a symbolic link to manage relative paths

os.makedirs("input_images",   exist_ok=True)
os.makedirs("raw_depth_output", exist_ok = True)
os.makedirs("depth_blur_results", exist_ok = True)

import warnings
warnings.filterwarnings('ignore')

ln: failed to create symbolic link './checkpoints/checkpoints': File exists


Load Depth Pro model

In [ ]:
import depth_pro

# Load model and preprocessing transform
model, transform = depth_pro.create_model_and_transforms(precision=torch.half)
model.to("cuda").eval()
print("Model Loaded Successfully...✅")

Model Loaded Successfully...✅


Obtain raw depth map and save the results

In [ ]:
def predict_depth(rgb_image: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    image = transform(rgb_image).to("cuda")
    prediction = model.infer(image)
    depth = prediction["depth"].detach().cpu().numpy().squeeze()
    inverse_depth = 1 / depth

    max_depth_vis = min(depth.max(), 1 / 0.1)
    min_depth_vis = max(depth.min(), 1 / 250)

    depth_clipped = np.clip(depth, min_depth_vis, max_depth_vis)

    depth_normalized =(depth_clipped - min_depth_vis) / max_depth_vis

    depth = (depth_normalized * 255)

    grayscale_depth = depth.astype(np.uint8)

    timestamp = int(time.time())
    filename = f"raw_depth_output/depth_map_{timestamp}.png"
    cv2.imwrite(filename, grayscale_depth)

    return grayscale_depth

### Download images used in demo

In [1]:
!wget https://learnopencv.com/wp-content/uploads/2025/04/Leopard-Cub.jpeg -O leopard.jpeg
!wget https://learnopencv.com/wp-content/uploads/2025/04/cave-scaled.jpg -O cave.jpg

--2025-04-04 03:05:18--  https://learnopencv.com/wp-content/uploads/2025/04/Leopard-Cub.jpeg
Resolving learnopencv.com (learnopencv.com)... 172.66.42.215, 172.66.41.41, 2606:4700:3108::ac42:2ad7, ...
Connecting to learnopencv.com (learnopencv.com)|172.66.42.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 631634 (617K) [image/jpeg]
Saving to: ‘leopard.jpeg’

leopard.jpeg        100%[===================>] 616.83K  --.-KB/s    in 0.03s   

2025-04-04 03:05:18 (19.5 MB/s) - ‘leopard.jpeg’ saved [631634/631634]

--2025-04-04 03:05:18--  https://learnopencv.com/wp-content/uploads/2025/04/cave-scaled.jpg
Resolving learnopencv.com (learnopencv.com)... 172.66.42.215, 172.66.41.41, 2606:4700:3108::ac42:2ad7, ...
Connecting to learnopencv.com (learnopencv.com)|172.66.42.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 678959 (663K) [image/jpeg]
Saving to: ‘cave.jpg’

cave.jpg            100%[===================>] 663.05K  --.-KB/s    in 0

## 3. Depth of Field using Gradio


Focal Plane (`focal_depth`): The imaginary plane where the image is perfectly in focus. Anything on this plane appears sharp, while objects in front or behind it become blurry.


Focus Range  (`min_focal_range`): The distance between the nearest and farthest objects that appear acceptably sharp in an image. It varies based on aperture, focal length, and subject distance.

---

The `apply_dof` function simulates a Depth of Field (DoF) effect by selectively blurring parts of an image based on a depth map. This mimics the way real cameras focus on a subject while blurring the background or foreground.

Let's understand step by step:

Uses a Gaussian function to compute how much each pixel should remain sharp.
1. Compute Sharpness Weights
```python
min_focal_range = 0.1  # Adjust as needed
sharpness_weights = np.exp(-((depth_map_normalized - focal_depth) ** 2) / (2 * min_focal_range ** 2))
```

  - Uses a Gaussian function to compute how much each pixel should remain sharp.
  - Pixels closer to focal_depth remain sharp, while others blur gradually.
  - `min_focal_range` controls the focus transition—lower values create a sharper transition.

2. Apply Gaussian Blur
```python
blurred_image = cv2.GaussianBlur(rgb_image, ksize = (51, 51), 0)
```
   -  Blurs the entire image using a Gaussian filter with a kernel size of (51, 51).

  - Acts as the out-of-focus version of the image.

3. Blend Sharp and Blurred Images
```python
sharpness_weights_3d = np.expand_dims(sharpness_weights, axis=2)
dof_image = sharpness_weights_3d * rgb_image + (1 - sharpness_weights_3d) * blurred_image
```
  -  Expands sharpness_weights to match the image dimensions (H × W × 3).

  - Blends the sharp and blurred images based on the weight mask.

4. Final Depth of Field Output
```python
dof_image = np.clip(dof_image, 0, 255).astype(np.uint8)
return dof_image
```
  - Ensures all pixel values are within the valid range `[0,255]`.

  - Converts the image to uint8 format for proper display.






In [ ]:
stored_rgb, stored_depth = None, None # global var

def apply_dof(rgb_image, focal_depth, raw_depth):

    # Converts the depth map to a [0,1] range for consistency.
    depth_map_normalized = cv2.normalize(src = raw_depth.astype(np.float32), dst = None, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX)

    min_focal_range = 0.10  # Adjust the focus range (region that will remain sharp) accordingly

    sharpness_weights = np.exp(-((depth_map_normalized - focal_depth) **2) / (2 * min_focal_range ** 2))
    sharpness_weights = sharpness_weights.astype(np.float32)

    blurred_image = cv2.GaussianBlur(src = rgb_image, ksize = (51, 51), sigmaX = 0)

    sharpness_weights_3d = np.expand_dims(sharpness_weights, axis = 2)

    dof_image = sharpness_weights_3d * rgb_image + (1 - sharpness_weights_3d) * blurred_image

    dof_image = np.clip(dof_image, 0, 255).astype(np.uint8)

    return dof_image

The following ultiltiy `process_image` processes an input image, computes its depth map, and applies a Depth of Field (DoF) effect dynamically.

In [ ]:
def process_image(rgb_image):
    global stored_rgb, stored_depth
    stored_rgb = rgb_image
    stored_depth = predict_depth(rgb_image)
    return stored_depth

# depth of field
def update_dof(focal_depth):
    return apply_dof(stored_rgb, focal_depth, stored_depth)

Finally putting this all together into an interactive Gradio UI.

The UI consists of **two tabs**:  

1. **Predict Depth**  
   - Upload an RGB image.  
   - Predict and display the raw depth map.  

2. **Depth of Field Effect**  
   - Visualize the DoF effect.  
   - Adjust the **focal depth** using a slider.  

---


Once the code runs, a **public URL** will be generated.  
Click on it to open the interface in a **new tab** and start playing with the DoF effect interactively!  


> **Disclaimer:**  
We have demonstrated an example using a **leopard image** and a scenic view, which correctly showcases the Depth of Field effect. However, **results may vary** for different images, as depth estimation accuracy depends on the scene complexity, lighting, and object structures.  


<div style="text-align: center;">
  <figure style="display: inline-block;">
    <video
       controls
       src="https://learnopencv.com/wp-content/uploads/2025/04/c0_simulating_depth_effect.webm"
       width="1080">
    </video>
    <figcaption>Gradio UI Demo</figcaption>
  </figure>
</div>



In [ ]:
import gradio as gr

initial_focal_value = 0 # slider default value at start

with gr.Blocks() as gui:
    with gr.Tab("Predict Depth"):
        with gr.Row():
            image_input = gr.Image(type="numpy", label="Upload RGB Image")
            depth_output = gr.Image(type="numpy", label="Predicted Raw Depth Map")
        process_button = gr.Button("Predict Depth")
        process_button.click(fn=process_image, inputs=image_input, outputs=depth_output)

    with gr.Tab("Depth of Field Effect"):
        with gr.Row():
            dof_output = gr.Image(type="numpy", label="Depth of Field Effect", width = 1920, height = 1080)
        with gr.Row():
            focal_slider = gr.Slider(0, 1, step = 0.01, value=initial_focal_value, label="Focal Depth Value")
            focal_slider.change(fn=update_dof, inputs=focal_slider, outputs=dof_output)

gui.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c2b94416ae2191a4fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://c2b94416ae2191a4fa.gradio.live


## 4. Run locally using OpenCV trackbar window

As we know, GUI-specific utilities do not work in the Colab environment. But if you want to truly feel the magic of this effect in real-time, try running it locally using an OpenCV trackbar!

For this, you will need the depth map output of the input image, which is saved under `./raw_depth_output`.

The real excitement kicks in when you slide the OpenCV trackbar—you’ll instantly see the depth effect shift smoothly with every tiny adjustment. Unlike Gradio, which introduces a slight transition delay, OpenCV renders changes instantaneously—letting you truly appreciate the subtle in-focus to out-of-focus transitions.

Just like in the leopard video shown at the start of the notebook, you’ll notice how the blur dynamically adjusts, pulling your focus from the foreground to the background in a way that feels almost surreal.

We highly recommend using the OpenCV version of this script—just follow the instructions and enjoy the thrill of depth control at your fingertips.

In [ ]:
# import cv2
# import numpy as np


# # Load RGB image and depth map
# rgb_image = cv2.imread("Leopard-Cub.jpeg")
# depth_map = cv2.imread("raw_depth_output/depth_map_1743586939.png", cv2.IMREAD_GRAYSCALE)


# # Normalize depth map to range [0, 1]
# depth_map_normalized = cv2.normalize(depth_map.astype(np.float32), None, 0, 1, cv2.NORM_MINMAX)

# # Get screen size and resize image to fit
# screen_width, screen_height = 1920, 1080  # Replace with your screen resolution if needed
# scale = min(screen_width / rgb_image.shape[1], screen_height / rgb_image.shape[0])
# new_width = int(rgb_image.shape[1] * scale)
# new_height = int(rgb_image.shape[0] * scale)

# rgb_image = cv2.resize(rgb_image, (new_width, new_height))
# depth_map_normalized = cv2.resize(depth_map_normalized, (new_width, new_height))

# # Function to apply depth of field effect
# def apply_dof(focal_depth):
#     focal_range = 0.1  # Range around focal depth to remain sharp

#     # Create smooth focus weights
#     sharpness_weights = np.exp(-((depth_map_normalized - focal_depth) ** 2) / (2 * focal_range ** 2))
#     sharpness_weights = sharpness_weights.astype(np.float32)

#     # Apply Gaussian blur to the background
#     blurred_image = cv2.GaussianBlur(rgb_image, (51, 51), 0)

#     # Blend the original image and blurred image using sharpness weights
#     sharpness_weights_3d = np.expand_dims(sharpness_weights, axis=2)  # Add a channel for blending
#     dof_image = sharpness_weights_3d * rgb_image + (1 - sharpness_weights_3d) * blurred_image
#     dof_image = np.clip(dof_image, 0, 255).astype(np.uint8)

#     return dof_image

# # Callback function for the trackbar
# def on_trackbar(val):
#     # Convert slider value (0-100) to focal depth (0.0-1.0)
#     focal_depth = val / 100.0
#     dof_image = apply_dof(focal_depth)
#     cv2.imshow(dof_image)

# # Create a window and resize it to fit the screen
# cv2.namedWindow("Depth of Field Effect", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Depth of Field Effect", new_width, new_height)

# # Create a trackbar (slider) at the top of the window
# cv2.createTrackbar("Focal Plane", "Depth of Field Effect", 50, 100, on_trackbar)  # Default at middle (50)

# # Show initial DOF effect
# initial_dof_image = apply_dof(0.5)  # Start with focal depth at 0.5
# cv2.imshow("Depth of Field Effect", initial_dof_image)

# # Wait until user closes the window
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## Conclusion  

Interesting right! Run this application through your **wildest examples**, experiment with different images, and let us know if you were able to **improve the application** or came up with a **unique and interesting idea**—we’d love to see what you create!  

